In [1]:
import ratemyprofessor
from tqdm import tqdm
import pandas as pd 

# Local

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from pathlib import Path


def get_professor_names(url):
    url = Path(url)

    with webdriver.Chrome() as driver:
        file_url = 'file://' + url.as_posix()
        driver.get(file_url)

        elements = driver.find_elements(By.CSS_SELECTOR, '.class-results-cell.instructor')

        professor_names = []
        for element in elements:
            try:
                # 尝试在当前教授元素中查找<a>标签
                link_element = element.find_element(By.TAG_NAME, 'a')
                # 获取<a>标签的文本，即教授的名字
                professor_name = link_element.text
                professor_names.append(professor_name)

            except NoSuchElementException:
                # 如果没有找到<a>标签，输出提示信息
                # print("未找到<a>标签在以下元素中:", element.text)
                pass
    
        return list(set(professor_names))

In [5]:
# url1 = "/Users/scott/Downloads/mat267_1.html"
# url2 = "/Users/scott/Downloads/mat267_2.html"
url2 = "/Users/scott/Downloads/mat267_ava.html"

# professor_names_23 = get_professor_names(url1)
# professor_names_24 = get_professor_names(url2)
professor_names_ava = get_professor_names(url2)

In [7]:
# all_mat_267_math_professor = list(set(professor_names_23 + professor_names_24))
all_mat_267_math_professor = professor_names_ava

In [8]:
asu = ratemyprofessor.get_school_by_name("Arizona State University")

In [9]:
# init all professors
professors = []
for name in tqdm(all_mat_267_math_professor):
    professors.append(ratemyprofessor.get_professor_by_school_and_name(asu, name))

100%|██████████| 4/4 [00:06<00:00,  1.57s/it]


In [10]:
# Get ranking data
professor_ranking_list = []
for professor in professors:
    if professor is None:
        continue
    if professor.school.name != "Arizona State University":
        continue
    professor_ranking_list.append([professor.name, professor.rating, professor.num_ratings, professor.school.name])

In [12]:
df = pd.DataFrame(professor_ranking_list)
df.columns = ['Name', 'Rating', 'RatingNum', 'School']
df.sort_values(by='Rating', ascending=False, inplace=True)
df
# df.to_excel("/Users/scott/Downloads/math_professor_ranking_ava.xlsx", index=False)

,Name,Rating,RatingNum,School
2,Tom Peebles,3.8,3,Arizona State University
1,Sergey Nikitin,2.9,263,Arizona State University
0,Joelle Park,2.3,6,Arizona State University
